<a href="https://colab.research.google.com/github/doudi25/Triton/blob/main/Fused_Softmax.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import triton
import triton.language as tl


In [ ]:
@triton.jit
def softmax_kernel(y_ptr,out_ptr,M,N,stride_ym,stride_yn
                   ,ROW_BLOCK:tl.constexpr,BLOCK_SIZE:tl.constexpr,
                   num_stages:tl.constexpr):
  program_id = tl.program_id(axis=0)
  offset_m = program_id * ROW_BLOCK + tl.arange(0,ROW_BLOCK)
  offset_n = tl.arange(0,BLOCK_SIZE)
  y_ptrs = y_ptr + offset_m[:,None] * stride_ym + offset_n[None,:] * stride_yn
  out_ptrs = out_ptr + offset_m[:,None] * stride_ym + offset_n[None,:] * stride_yn
  mask = offset_n < N
  y = tl.load(y_ptrs,mask=mask[None,:],other=-float('inf'))
  y_max = tl.max(y,axis=1)
  exp = tl.exp(y - y_max[:,None])
  sum = tl.sum(exp,axis=1)
  out = exp / sum[:,None]
  tl.store(out_ptrs,out,mask=mask[None,:])



In [ ]:
def softmax(y):
  assert y.is_contiguous()
  M, N = y.shape
  out = torch.empty_like(y,device=y.device)
  BLOCK_SIZE = triton.next_power_of_2(N)
def softmax(y):
  assert y.is_contiguous()
  M, N = y.shape
  out = torch.empty_like(y,device=y.device)
  BLOCK_SIZE = triton.next_power_of_2(N)
  grid = lambda meta: (triton.cdiv(M,4),)
  num_stages = 2
  softmax_kernel[grid](y,out,M,N,y.stride(0),y.stride(1),ROW_BLOCK=4,BLOCK_SIZE=BLOCK_SIZE,num_stages=num_stages)
  return out

In [ ]:
torch.manual_seed(0)
x = torch.randn(1823, 781, device='cuda')
y_triton = softmax(x)
y_torch = torch.softmax(x, axis=1)
assert torch.allclose(y_triton, y_torch), (y_triton, y_torch)